In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('./pdata')

In [ ]:
datarps2 = data.drop(['Unnamed: 0'], axis=1)
datarps2.plot()

In [ ]:
#Conver to Array
tdata = []
for i in range(datarps2.shape[0]):
  tdata.append([float(datarps2.iloc[i]['CPU']), float(datarps2.iloc[i]['RPS']), float(datarps2.iloc[i]['MEM'])])

In [ ]:
TX = np.array(tdata)

In [ ]:
#Create window of last 5 mins
def create_windows(data, windsize = 5):
  wdata = []
  ydata = []
  for i in range(windsize, len(data)):
    idata = []
    for j in range(windsize, 0, -1):
      idata.append(data[i - j])
    wdata.append(idata)
    ydata.append(data[i][0])
  return np.array(wdata), np.array(ydata)

In [ ]:
tx, yx = create_windows(TX)

In [ ]:
from keras.layers import Bidirectional, Dense, Dropout
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout,BatchNormalization

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=5, return_sequences=True, activation="elu"), input_shape=( 5, 3)))
model.add(Bidirectional(LSTM(units=3, activation="elu")))
model.add(Dense(units=1, activation="elu"))
model.build(input_shape=(None,  5, 3))

In [ ]:
adamOpt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
model.compile(loss='mean_squared_error', optimizer=adamOpt, metrics=['mae'], run_eagerly=True)

In [ ]:
history = model.fit(trainX, trainY, validation_split=0.2, epochs=40, batch_size=64, verbose=2, shuffle=False)

In [ ]:
#Save Model
import pickle
model.save_weights("bilstm_model.h5")

In [ ]:
testPredict = model.predict(testX)

In [ ]:
import matplotlib.pyplot as plt
import pickle

scaler1 = pickle.load(open('./scc.pkl', 'rb'))

TestY= pd.DataFrame(testY,columns=['avg_cpu'])
PredY=pd.DataFrame(testPredict,columns=['avg_cpu'])
testY = scaler1.inverse_transform(TestY)
testPredict = scaler1.inverse_transform(PredY)

TestY= pd.DataFrame(testY,columns=['avg_cpu'])
PredY=pd.DataFrame(testPredict,columns=['avg_cpu'])

x = [i for i in range(0,testPredict.shape[0])]

fig, axs = plt.subplots(1,figsize=(10,15))

axs.plot(x,TestY.avg_cpu,'-.',label='CPU',color='green')
axs.plot(x,PredY.avg_cpu,'--',label='Pred CPU',color='black')

plt.xlabel('Time in Minutes') 
plt.ylabel('CPU Usage %') 
  
# displaying the title
plt.title("CPU prediction")

axs.legend()

In [ ]:
#Metrics
y_actual, y_predicted = testY, testPredict

from sklearn.metrics import mean_squared_error
import math
 
MSE = mean_squared_error(y_actual, y_predicted)
 
RMSE = math.sqrt(MSE)

print("RMSE: ",RMSE)

from sklearn.metrics import mean_absolute_percentage_error

print("MAPE: ", mean_absolute_percentage_error(y_actual, y_predicted))

from sklearn.metrics import mean_absolute_error

print("MAE: " mean_absolute_error(y_actual, y_predicted, multioutput='raw_values'))